In [0]:
import numpy as np
from operator import itemgetter
import copy
import uuid

tablero_objetivo = np.array([[1,2,3],
                             [4,5,6],
                             [7,8,0]])
tablero_dado = np.array([[1,8,2],
                         [0,4,3],
                         [7,6,5]])

In [0]:
global nodos_abiertos
nodos_abiertos = []
global nodos_cerrados
nodos_cerrados = []

In [0]:
def calcular_h(h, c=1):
  f = h + c
  return f

In [0]:
def obtenerDistintos(mat1, mat2=tablero_objetivo):
  diff = np.setdiff1d(mat1, mat2)
  return len(diff)

In [0]:
def getNodeStart():
  x, y = tablero_dado.shape

  for i in range(x):
    for j in range(y):
      if tablero_dado[i,j] == 0:
        return [i,j]
  return [0,0]

In [0]:
def getUniqueId():
  return uuid.uuid4()

def printMats(mat):
  h,w = mat.shape
  for i in range(h):
    print(mat[i])
  print('\n')

In [0]:
def mover(direct, nodo):
  #Copiamos nodo
  local_nodo = nodo.copy()

  #Posición de hueco
  x = local_nodo[0][0]
  y = local_nodo[0][1]

  #Variable para poder guardar la posición futura del hueco
  coord = [x,y]

  #Copiamos el tablero del nodo para plasmar el movimiento
  mat = local_nodo[1].copy()


  #Si es a la derecha
  if direct == 'r':
    cell = mat[x,y+1]
    mat[x,y+1] = mat[x,y]
    mat[x,y] = cell
    coord[0] = x
    coord[1] = y+1

  #Si es a la izquierda
  elif direct == 'l':
    cell = mat[x,y-1]
    mat[x,y-1] = mat[x,y]
    mat[x,y] = cell
    coord[0] = x
    coord[1] = y-1

  #Si es arriba
  elif direct == 't':
    cell = mat[x-1,y]
    mat[x-1,y] = mat[x,y]
    mat[x,y] = cell
    coord[0] = x-1
    coord[1] = y

  #Si es abajo
  elif direct == 'd':
    cell = mat[x+1,y]
    mat[x+1,y] = mat[x,y]
    mat[x,y] = cell
    coord[0] = x+1
    coord[1] = y

  #Calculamos la heurística para nuetro nuevo nodo
  heur = calcular_h(h=obtenerDistintos(mat), c=1) + local_nodo[2]

  #Creamos el nodo
              #pos   #mat        #h    #padre         #id   
  new_nodo = [coord, mat.copy(), heur, local_nodo[4], getUniqueId()]
  

  #Si el caso del tablero ya existe en abiertos pero su coste es menor al del actual pasamos para poder ir a ese
  for n in nodos_abiertos:
    if np.array_equal(n[1],new_nodo[1]):
      if n[2] <= heur + local_nodo[2]:
          return
  #Sino, si el caso existe en cerrados pero su coste es menor al actual pasamos porque ya lo hemos visitado
  for n in nodos_cerrados:
    if np.array_equal(n[1],new_nodo[1]):
      if n[2] <= heur + local_nodo[2]:
          return
  #Sino lo añadimos a abiertos
  nodos_abiertos.append(new_nodo)


In [0]:
def expandir_nodo(nodo):
  #Copiamos nodo
  local_nodo = nodo.copy()

  #Obtenemos posición del hueco
  x = local_nodo[0][0]
  y = local_nodo[0][1]

  #Si es 0,0 || 0,1 || 0,2
  if x == 0:
    if y == 0:
      #moverDerecha
      mover('r', local_nodo)
      #moverAbajo
      mover('d', local_nodo)
    elif y == 1:
      #moverDerecha
      mover('r', local_nodo)
      #moverAbajo
      mover('d', local_nodo)
      #moverIzquierda
      mover('l', local_nodo)
    elif y == 2: 
      #moverIzquierda
      mover('l', local_nodo)
      #moverAbajo
      mover('d', local_nodo)

  #Si es 1,0 || 1,1 || 1,2
  elif x == 1:
    if y == 0:
      #moverArriba
      mover('t', local_nodo)
      #moverDerecha
      mover('r', local_nodo)
      #moverAbajo
      mover('d', local_nodo)
    elif y == 1:
      #moverDerecha
      mover('r', local_nodo)
      #moverArriba
      mover('t', local_nodo)
      #moverAbajo
      mover('d', local_nodo)
      #moverIzquierda
      mover('l', local_nodo)
    elif y == 2: 
      #moverIzquierda
      mover('l', local_nodo)
      #moverAbajo
      mover('d', local_nodo)
      #moverArriba
      mover('t', local_nodo)

  #Si es 2,0 || 2,1 || 2,2
  elif x == 2:
    if y == 0:
      #moverArriba
      mover('t', local_nodo)
      #moverDerecha
      mover('r', local_nodo)
    elif y == 1:
      #moverDerecha
      mover('r', local_nodo)
      #moverArriba
      mover('t', local_nodo)
      #moverIzquierda
      mover('l', local_nodo)
    elif y == 2: 
      #moverIzquierda
      mover('l', local_nodo)
      #moverArriba
      mover('t', local_nodo)
  

In [0]:
#Calculo de la herística
heuristica = calcular_h(h=obtenerDistintos(tablero_dado), c=1)

#Creamos el nodo inicial
              #pos            #mat                 #heu       #p #id 
node_start = [getNodeStart(), tablero_dado.copy(), heuristica ,0, 0]

#Añadimos nodo inicial a abiertos
nodos_abiertos.append(node_start)

In [393]:
global nodo_local

#Mientras haya nodos
while len(nodos_abiertos) != 0:

  #ordenar lista de menor a mayor f
  nodos_abiertos = sorted(nodos_abiertos, key=lambda row: row[2])

  #Obtener nodo con menor f
  nodo_local = nodos_abiertos.pop(0).copy()

  #Comprobar si es el objetivo
  if np.array_equal(tablero_objetivo, nodo_local[1]):
    print("Solved")
    break

  #Expandimos nodo
  expandir_nodo(nodo_local)

  #Una vez expandido y preocesado guardamos padre en cerrados
  nodos_cerrados.append(nodo_local.copy())

Solved


In [394]:
camino = []
camino.append(nodo_local)

padre = nodo_local[3]
while padre != 0:
  for n in nodos_cerrados:
    if n[4] == padre:
      camino.append(n)
      padre = n[3]

camino.reverse()

print("<<< Tablero dado >>>")
printMats(tablero_dado)

print("<<< Objetivo >>>")
printMats(tablero_objetivo)

print("<<< Camino >>>")
for n in camino:
  print("f = {}".format(n[2]))
  printMats(n[1])

<<< Tablero dado >>>
[1 8 2]
[0 4 3]
[7 6 5]


<<< Objetivo >>>
[1 2 3]
[4 5 6]
[7 8 0]


<<< Camino >>>
f = 2
[1 8 2]
[4 0 3]
[7 6 5]


f = 3
[1 0 2]
[4 8 3]
[7 6 5]


f = 4
[1 2 0]
[4 8 3]
[7 6 5]


f = 5
[1 2 3]
[4 8 0]
[7 6 5]


f = 6
[1 2 3]
[4 8 5]
[7 6 0]


f = 7
[1 2 3]
[4 8 5]
[7 0 6]


f = 8
[1 2 3]
[4 0 5]
[7 8 6]


f = 9
[1 2 3]
[4 5 0]
[7 8 6]


f = 10
[1 2 3]
[4 5 6]
[7 8 0]


